In [14]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from vacationkey import google_key
gmaps.configure(api_key=google_key)

In [15]:
#import cities.csv from part 1
cities_df = pd.read_csv('cities.csv')
cities_df.count()

City          546
Lat           546
Lng           546
Max Temp      546
Humidity      546
Cloudiness    546
Wind Speed    546
Country       543
Date          546
dtype: int64

In [16]:
cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,zaraza,9.35,-65.32,74.95,95,100,3.44,VE,1597785387
1,beloha,-25.17,45.05,68.85,60,10,8.39,MG,1597785387
2,mayumba,-3.43,10.66,73.47,90,100,10.69,GA,1597785323
3,atuona,-9.80,-139.03,79.16,74,2,21.23,PF,1597784921
4,punta arenas,8.99,125.34,76.59,85,100,4.29,PH,1597785388


In [19]:
locations = cities_df[['Lat','Lng']]
humidity = cities_df['Humidity']

#create a gmap figure
fig = gmaps.figure()

#add the heatmap layer
fig.add_layer(gmaps.heatmap_layer(locations,weights=humidity,dissipating=False, max_intensity=100, point_radius=2))
fig

Figure(layout=FigureLayout(height='420px'))

In [23]:
#new df fitting ideal parameters
#A max temperature lower than 75 degrees but higher than 65.
#Wind speed less than 10 mph.
#Zero cloudiness.
#Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

ideal_df = cities_df.loc[(cities_df['Max Temp'] < 85) & (cities_df['Max Temp']>=75)\
                         & (cities_df['Wind Speed'] < 10)\
                         & (cities_df['Cloudiness'] == 0)] 

#store as a new variable
hotel_df = ideal_df.copy()

hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
30,sioux lookout,42.73,-96.23,82.99,49,0,1.99,US,1597785392
54,clyde river,41.30,-82.98,77.00,45,0,1.01,US,1597785397
104,burdur,37.72,30.29,77.00,41,0,3.36,TR,1597785406
153,misratah,32.38,15.09,82.20,71,0,5.44,LY,1597785417
169,suez,29.97,32.53,76.98,74,0,9.42,EG,1597785421
266,hami,42.80,93.45,79.09,17,0,5.66,CN,1597785248
292,mallaig,54.22,-111.37,83.66,47,0,5.64,CA,1597785446
312,salto,-31.38,-57.97,75.20,23,0,4.81,UY,1597785300
344,marsa matruh,35.88,14.50,81.00,69,0,8.05,MT,1597785456
364,awjilah,29.11,21.29,83.21,38,0,8.90,LY,1597785460


In [24]:
#set parameters to search for hotels within 5000 mile radius
#https://developers.google.com/places/web-service/search for parameters

hotel_df["Hotel Name"] = ""

hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
30,sioux lookout,42.73,-96.23,82.99,49,0,1.99,US,1597785392,
54,clyde river,41.30,-82.98,77.00,45,0,1.01,US,1597785397,
104,burdur,37.72,30.29,77.00,41,0,3.36,TR,1597785406,
153,misratah,32.38,15.09,82.20,71,0,5.44,LY,1597785417,
169,suez,29.97,32.53,76.98,74,0,9.42,EG,1597785421,
266,hami,42.80,93.45,79.09,17,0,5.66,CN,1597785248,
292,mallaig,54.22,-111.37,83.66,47,0,5.64,CA,1597785446,
312,salto,-31.38,-57.97,75.20,23,0,4.81,UY,1597785300,
344,marsa matruh,35.88,14.50,81.00,69,0,8.05,MT,1597785456,
364,awjilah,29.11,21.29,83.21,38,0,8.90,LY,1597785460,


In [31]:
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    
#iterate through the rows of the df
for index, row in hotel_df.iterrows():
    
    coordinates = f"{row['Lat']},{row['Lng']}"
    params = {"location": coordinates, "radius": 1000, "type": 'lodging', "key": google_key}
    response = requests.get(base_url, params=params).json()
    try:
        hotel_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
    except(KeyError, IndexError):
        print('Missing Field. Skipping...')

Missing Field. Skipping...
Missing Field. Skipping...
Missing Field. Skipping...
Missing Field. Skipping...


In [32]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
30,sioux lookout,42.73,-96.23,82.99,49,0,1.99,US,1597785392,Holiday Inn Express & Suites Le Mars
54,clyde river,41.30,-82.98,77.00,45,0,1.01,US,1597785397,Clyde Moose Lodge
104,burdur,37.72,30.29,77.00,41,0,3.36,TR,1597785406,Özeren Grand Hotel & Spa
153,misratah,32.38,15.09,82.20,71,0,5.44,LY,1597785417,فندق الماسة
169,suez,29.97,32.53,76.98,74,0,9.42,EG,1597785421,Elherek
266,hami,42.80,93.45,79.09,17,0,5.66,CN,1597785248,Post And Telecommunication Hotel
292,mallaig,54.22,-111.37,83.66,47,0,5.64,CA,1597785446,
312,salto,-31.38,-57.97,75.20,23,0,4.81,UY,1597785300,Hotel Tia
344,marsa matruh,35.88,14.50,81.00,69,0,8.05,MT,1597785456,Tal-Bahar
364,awjilah,29.11,21.29,83.21,38,0,8.90,LY,1597785460,استراحة التعاون أوجلة


In [33]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [34]:
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))